In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
train=pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [ ]:
avgs=pd.DataFrame(train.drop(['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'ts_id'],
                              #'resp', 'feature_0', 'feature_19', 'feature_51', 'feature_56', 'feature_58',
#           'feature_59', 'feature_60', 'feature_62', 'feature_66', 'feature_67', 'feature_70',
#            'feature_89', 'feature_96', 'feature_102', 'feature_119', 'feature_121', 'feature_122', 'feature_124', 'feature_127',
#           'ts_id']
                               axis=1).mean())

#avgs=pd.DataFrame(train.mean())

avgs=avgs.pivot_table(columns=list(avgs.index))

In [ ]:
for i in train.columns[train.isnull().any(axis=0)]:
    train[i].fillna(train[i].mean(), inplace=True)

In [ ]:
ids=train[['ts_id']]
X=train.drop(['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'feature_0', 'feature_19', 'feature_51', 'feature_56', 'feature_58',
           'feature_59', 'feature_60', 'feature_62', 'feature_66', 'feature_67', 'feature_70', 
            'feature_89', 'feature_90', 'feature_95', 'feature_96', 'feature_102', 'feature_119', 'feature_121', 'feature_122', 'feature_124', 'feature_127',
            'feature_129', 'ts_id'], axis=1)
Y=train[['resp']]

In [ ]:
# Create Parameters
estimators=[100]
tree_depths=[7,8]
learning_rates=np.arange(.32,.48,.02)
subsamples=[.7, .8]
colsamples=[.8, .9,1.0]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2)

In [ ]:
#from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as mse
import xgboost as xgb 
from tqdm import tqdm


xgb_mod=xgb.XGBRegressor(objective='reg:squarederror',
                         n_estimators=100, seed=123,
                         subsample=.8,
                         colsample_bytree=.9,
                         eta=.4,
                         tree_method='hist',
                         #gpu_id=0,
                         max_depth=8,
                        )
xgb_mod.fit(X_train, Y_train)

y_pred=xgb_mod.predict(X_test)
predictions=[(round(value) for value in y_pred)]
rmse=np.sqrt(mse(Y_test, y_pred))

#print('learning rate: % f' %(learning_rates[i]))
print('RMSE: % f' %(rmse))

#del X_train
#del X_test
#del Y_train
#del Y_test

In [ ]:
train['resp_pred']=xgb_mod.predict(X)

returns=train[['date', 'weight', 'resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']].copy()

returns['max']=returns[['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']].max(axis=1)
returns['min']=returns[['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']].min(axis=1)
returns['spread']=(returns['max']-returns['min'])/2
spread_df=pd.DataFrame(returns['spread'])

df2=pd.concat([train, spread_df], axis=1)

del train
del spread_df

In [ ]:
x=df2[['feature_43', 'feature_42', 'feature_126', 
         'feature_124', 'feature_120', 'feature_128']]
y=df2[['spread']]

mean_spread=y.mean()

In [ ]:
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)

In [ ]:
#xgb_mod_spread=xgb.XGBRegressor(objective='reg:squarederror',
#                         n_estimators=100, seed=123,
#                         subsample=1,
#                         eta=.43,
#                         tree_method='hist',
#                         #gpu_id=0,
#                         max_depth=6,
#                        )
#xgb_mod_spread.fit(x_train, y_train)


#lm_vol=LinearRegression()
#lm_vol.fit(x_train, y_train)
#y_pred=xgb_mod_spread.predict(x_test)
#mean_spread_df=pd.DataFrame(columns=['y_pred'])
y_pred=mean_spread
#predictions=[(round(value) for value in y_pred)]
#rmse_vol=pd.DataFrame([np.sqrt(mse(y_test, y_pred))], columns=['rmse'])
#rmse_vol

In [ ]:
#del train
del returns
#del xgb_mod
del x_train
del x_test
del y_train
del y_test
del X
del Y

In [ ]:
#df2['spread_pred']=xgb_mod_spread.predict(x)
df2['spread_pred']=mean_spread
#del lm_vol
del x
del y

In [ ]:
feature_names=['weight', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 
              'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 
              'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 
              'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 
              'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 
              'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 
              'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 
              'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 
              'feature_66', 'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feature_71', 'feature_72', 'feature_73', 'feature_74', 'feature_75', 
              'feature_76', 'feature_77', 'feature_78', 'feature_79', 'feature_80', 'feature_81', 'feature_82', 'feature_83', 'feature_84', 'feature_85', 
              'feature_86', 'feature_87', 'feature_88', 'feature_89', 'feature_90', 'feature_91', 'feature_92', 'feature_93', 'feature_94', 'feature_95', 
              'feature_96', 'feature_97', 'feature_98', 'feature_99', 'feature_100', 'feature_101', 'feature_102', 'feature_103', 'feature_104', 'feature_105', 
              'feature_106', 'feature_107', 'feature_108', 'feature_109', 'feature_110', 'feature_111', 'feature_112', 'feature_113', 'feature_114', 'feature_115', 
              'feature_116', 'feature_117', 'feature_118', 'feature_119', 'feature_120', 'feature_121', 'feature_122', 'feature_123', 'feature_124', 'feature_125', 
              'feature_126', 'feature_127', 'feature_128', 'feature_129', 'date']

In [ ]:
mean_spread=mean_spread.values[0]

In [ ]:
%%time
from tqdm import tqdm


import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in tqdm(iter_test):
    test_df=pd.concat([avgs, test_df], axis=0).ffill(axis=0).iloc[[1]]
    test_df=test_df[feature_names]
    
    #test_df=np.array(test_df)
    #avgs=np.array(avgs)
    #test_df[np.isnan(test_df)]=np.nanmean(avgs)
    
    #for i in range(len(test_df.columns)):
    #    test_df.iloc[:, i]=test_df.iloc[:, i].astype(float)
    
    #spread_pred=xgb_mod_spread.predict(test_df[['feature_43', 'feature_42', 'feature_126', 
    #                                            'feature_124', 'feature_120', 'feature_128']])[0]
    resp_pred=xgb_mod.predict(test_df.drop(['date', 'weight', 'feature_0', 
                                                    'feature_19', 'feature_51', 'feature_56', 'feature_58',
                                                    'feature_59', 'feature_60', 'feature_62', 'feature_66', 
                                                    'feature_67', 'feature_70', 'feature_89','feature_90',
                                                    'feature_95', 'feature_96', 
                                                    'feature_102', 'feature_119', 'feature_121', 'feature_122',
                                                    'feature_124', 'feature_127', 'feature_129'], axis=1))[0]
    
    #decision=np.where((resp_pred-(.05*abs(mean_spread))) > 0, 1, 0)
    #print(decision)
    decision=np.where((resp_pred) > 0,1,0)
    sample_prediction_df.action = int(decision) #make your 0/1 prediction here
    
    #sample_prediction_df.action = np.where((test_df['resp_pred']-(.25*test_df['spread_pred'].abs())) > 0, 1, 0)#make your 0/1 prediction here
    #sample_prediction_df.action = decision[0].int()
    
    env.predict(sample_prediction_df)